# Lab 3: PID Station Keeping and Disturbance Rejection

Type name here

Monday 1:25pm section

Table #

## Contents:

1. [Pre-lab Questions](#prelab)
1. [Lab Procedure](#lab)
1. [Post-lab Questions](#postlab)
1. [Submissions](#sub)

## Learning Objectives

1. Implement P, PI, and PID control laws on motor-propeller systems
2. Determine the effects of controller parameters on closed-loop station keeping and disturbance rejection

<a id='prelab'></a>

## Pre-lab Questions

In this lab, you will design feedback controllers to balance a pinned arm using a propeller at the tip, as shown in the following figure.

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab3/Rotating%20arm.png?raw=true" width="600" />

This system is inherently unstable: the arm will not remain horizontal unless there is a thrust force to balance the moment due to gravity.  The equations of motion (EOMs) can be found by taking a sum of moments about the pin (axis of rotation):

$\sum M_0 = J \ddot{\theta} = -b \dot{\theta} - mgR_0\cos(\theta) + TR$

which, upon rearranging, gives

$J \ddot{\theta} + b \dot{\theta} + mgR_0\cos(\theta) = TR$

This is a non-linear second-order ODE.  Moreover, the thrust $T$ is dependent on the distance $d$ to the ground, which is a function of the angle of the arm $\theta$.  Indeed, the closer the fan is to the ground, the more thrust it generates for a given fan speed.  More details about this system's dynamics are given in the Lab 3 simulator notebook.

In this lab, you will utilize PID control laws for **station keeping** and **disturbance rejection**.  For station keeping, you have a "set point" of operation in mind, and you want to keep the system near that set point depite outside disturbances, noise, and/or uncertainty.  Imagine trying to draw a straight line while someone is shaking your arm.  This can be quantitatively defined as "staying between the lines", where the lines represent the maximum allowable error, or perhaps one or more standard deviations of allowable error:

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab3/Station%20keeping.png?raw=true" width="600" />

Feedback control, like PID, is necessary because it is impossible to predict the random disturbances the system might face.  For example, the automatic flight control in an airplane is able to maintain the aircraft in a horizontal orientation dispite random turbulence it may be subjected to.

Disturbance rejection means restoring the system to equilibrium (or within some bounds of equilibrium) quickly and efficiently after a random, unpredictable disturbance:

<img src="https://github.com/AdaWick/MEEG312/blob/master/Lab3/Disturbance%20rejection.png?raw=true" width="600" />

Impacts, large gusts of wind, and sudden accelertions are all types of disturbances.  For measuring the quality of disturbance rejection, we will use the same measures as Lab 2 for the step response: things like rise time, settling time, and overshoot.

For measuring the quality of station keeping, we will need a new calculation.  Specifically, we will use a common measure called the [root-mean square (RMS)](https://en.wikipedia.org/wiki/Root_mean_square) of the error, where, as in Lab 2, the error is the difference between where the system is and where we want it to be.  The formula for RMS is

$e_{RMS} = \sqrt{\dfrac{1}{n}(e^2[0] + e^2[1] + \dotsb + e^2[n])}$,

where $n$ is the total number of data points (i.e. simulation time steps).

The RMS error is always positive and can only be 0 if the error is exactly 0 for all time steps.

**Problem 1 (10 points):**

Add code to compute the RMS error of the system's position vs. time loaded from the provided data set `test3.txt`.  This is an actual data set recorded in lab.  Position is measured in volts measured by the Hall effect sensor, which measures the distance of the back end of the pivoting arm from the ground.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss

In [ ]:
# read data from text file, skipping first 2 rows and last row, ignore ';'
data = np.genfromtxt('test3.txt',skip_header=1)
t = data[:,0]     # time is column 1 (recall column numbering starts at 0)
r = data[:,2]     # desired (reference) position is column 3
x = data[:,3]     # actual position is column 4
plt.plot(t,r,t,x)

# compute RMS error (input your code here)


**Problem 2 (10 points):**

In the following blank cell, compute the "baseline" RMS error, i.e. the RMS error before the first disturbance.  Your code should automatically detect when the disturbance occurs (you can decide on the threshold), and only compute the RMS on the data before this happens.

<a id='lab'></a>

## Lab Procedure

In this lab and in Lab 4, you will use the myVTOL National Instruments (NI) miniSystem by Pitsco Education, the NI myDAQ data acquisition device, and NI LabVIEW software to design a  to hover, track trajectories, and reject disturbances. The myVTOL system uses a variable-speed PC cooling fan to create the lift for the one degree of freedom (DOF) system. The angle of the arm the fan is mounted on is measured using an analog Hall Effect sensor that responds to a changing magnetic field by varying voltage output.

**In-person labs:**

1. On a flat surface, plug the myVTOL (green circuit board) into the myDAQ (beige box).  The connector can only fit one way, and the connector will be flush with the body of the DAQ when properly connected.
2. Plug the AC adaptor into a power outlet and connect the to the myVTOL board.  (Do not use the 9V battery.)
3. Set the switch on the myVTOL to ON.
4. Connect the myDAQ to the computer using the USB cable.  The blue LED should light up on the myDAQ next to the USB cable port.
5. Copy the file `MEEG312 Lab 3.llb` from Canvas to the computer's desktop, if it is not already there.  Double-click it to open it.  This library contains all of the LabView Virtual Instruments (VIs) we will use for this lab.

**Online labs**

Open the file "Lab 3 simulator.ipynp" in a separate tab.

1. This file is divided into two parts, Part 1 is Manual Control and Part 2 is Automatic Control.  These parts correspond to the in-person lab Parts.

2. In Part 1, you can control the system in real time using your mouse.  You can stop the control at any time by clicking the stop button next to the animation.

3. In Part 2, input the parameters for each experiment.  In particular, you will be changing the controller gains according the procedures of Part 2 below.

4. In Parts 1-2, optionally save an image of the plot of the arm's motion.

5. In Parts 1-2, run the last cell to save your data into a text file.  It will prompt you for a file name.  You should not add the ".txt" to the file name; this will be added automatically.  **Be sure to do this after every simulation you want to save.  If you don't, your data will be overwritten.**

### Part 1: Manual Control

First, you will try to stabilize and track a sinusoidal trajectory manually, using your mouse or keyboard to change the motor voltage.  This will give you an idea how difficult controlling a barely stable system can be!

1. Open the VI `MEEG312 - Manual Control.vi`.
1. Click the white arrow in the upper left hand corner of the window to run the VI.
1. Use the page up/page down keys or use the mouse to slide the manual fan power control level up and down to control the output voltage to the fan and get the Hall effect sensor voltage within the target range.  The arm should move easily to the top of its range of motion if you give it the max voltage (10 V).  You  may need to wiggle the arm if it gets stuck against the support and/or adjust the position of the counterweight.
1. Try to balance the fan arm so that it is parallel to the table.  Copy the "Sensor Reading" voltage into the "Target Sensor Reading" voltage box.
1. Give the fan arm a small disturbance.  Note how it recovers (or doesn't).  Press the "Stop" button to choose a filename and save.  Don't worry if there is data in the file you don't want to plot later; you can always trim the data later in Python.
1. Now click the moving target button.  Try to manually control the fan so that it tracks the given sinusoid.  The indicator will turn bright green when you’re in range.
1. Try to capture three "good" (or "pretty good") cycles of oscillation while tracking the sinusoid as best you can.  Each person should try this and save their results.  **Briefly reflect on the challenges of manually controlling this system.**
1. When you're finished with this part, close the VI.

### Part 2: PID Control

In this part, you will design a PID controller to hold a given angle (hover).  While doing this, you will test the stability and evaluate the closed-loop performance by delivering manual disturbances to the system.

1. Open the VI `MEEG312 - PIDControl.vi`.
1. Set all of the "PID Gain Settings" to 0 to start.
1. Set the "Control Mode" button to "Manual".  Run the VI by clicking the white arrow in the upper left hand corner.  Adjust the fan voltage until the arm is parallel to the table.  Again, you may need to wiggle the arm so that it rotates smoothly.
1. Copy the "Sensor Reading" voltage into the "Target Sensor Reading" voltage box.
1. Now, switch the "Control Mode" button to "PID".  Gradually increase the Proportional gain.  **Explain what happens as you increase the gain in terms of steady-state error.  Try deflecting the arm and watch the response.**
1. Save a few trials at different proportional gains by hitting the "Stop" button and saving the data.  **Record your gain settings because the data file does not save them.**
1. Now first try increasing the Integral gain and then the Derivative gain.  You will probably need a small Integral gain and a large Derivative gain compared to your Proportional gain.
1. For each gain setting, give the arm a small tap and see how it responds.  You will use these transient responses to characterize your controller in terms of rise time, overshoot, etc.
1. Again, save a few trials for different gain combinations by hitting the "Stop" button and saving the data.  **Record your gain settings because the data file does not save them.**
1. When you're finished with this part, close the VI.

**Turn off the myVTOL and unplug everything once you are finished.  Put the system back in the black case and the instructor/TA will collect it.**

<a id='postlab'></a>

## Post-lab Questions

1. For the manual control case (Part 1), describe the challenges in controlling the system.  Include plots of your personal trial.  Describe what you see and match that to your experience controlling the system.
2. Describe how varying the gains -- proportional, integral, and derivative -- of the controller affects its closed-loop response to disturbances.  Use as many plots and trials as needed to sufficiently support your discussion.

At the end, please write a conclusion paragraph (or two) summarizing your results and outlining how a PID controller could be designed for this myVTOL system.

<a id='sub'></a>

## Submissions

Please submit the following on Canvas (one submission per team):

1. Your completed Jupyter notebook (this file)
2. All data (.txt) files
3. All screen captures (they can be embedded in the Jupyter notebook or saved as separate files)

Please label your data files and screen captures in a logical manner so that they can be correlated.